# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Reading in csv
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data = weather_data[["City", "Lat", "Lng", "Max Temp", 
                             "Humidity", "Cloudiness", "Wind Speed",
                             "Country", "Date"]]
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cherskiy,68.7500,161.3000,-24.23,86,100,7.36,RU,1611518356
1,Lavrentiya,65.5833,-171.0000,0.57,85,100,2.59,RU,1611518356
2,Ushuaia,-54.8000,-68.3000,64.40,32,40,28.77,AR,1611518356
3,Mount Gambier,-37.8333,140.7667,71.60,46,95,5.75,AU,1611518356
4,Karachi,24.9056,67.0822,53.60,24,0,11.50,PK,1611518356


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configuring gmaps key
gmaps.configure(api_key=g_key)

In [4]:
# Creating heatmap
locations = weather_data[["Lat", "Lng"]]
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weather_data["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Filtering data by ideal conditions
ideal_weather_df = weather_data.loc[(weather_data["Max Temp"] > 70) & (weather_data["Max Temp"] < 80) & 
                                    (weather_data["Wind Speed"] < 8) & (weather_data["Cloudiness"] == 0)]
ideal_weather_df.dropna()
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,São Filipe,14.8961,-24.4956,72.91,69,0,7.43,CV,1611518357
66,Kindia,10.0569,-12.8658,74.73,21,0,3.38,GN,1611518425
138,Jojoima,7.8792,-10.7878,75.61,34,0,2.89,SL,1611518559
144,Dingle,10.9995,122.6711,74.91,92,0,6.78,PH,1611518560
201,Boueni,-12.9025,45.0761,77.00,88,0,2.30,YT,1611518634
250,Nong Kung Si,16.6500,103.3000,73.00,99,0,3.85,TH,1611518703
343,Bouloupari,-21.8667,166.0500,78.80,73,0,6.91,NC,1611518838
520,Oussouye,12.4850,-16.5469,78.80,65,0,7.85,SN,1611519522
550,Kampot,10.6167,104.1833,75.20,88,0,2.30,KH,1611519527


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Creating blank column to store hotel name
hotel_df = ideal_weather_df
hotel_df["Hotel Name"] = ""

<ipython-input-6-72d632eeed43>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


In [8]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    
    base_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    results = requests.get(base_url, params=params)
    
    results = results.json()
    print(f"Fetching {row['City']}")
    try:
        hotel_df.loc[index, "Hotel Name"] = results["results"][0]["name"]
    
    except (KeyError, IndexError):
        # If there is no hotel in range, an Index Error typically occurs
        # Store hotel as "No hotel in range"
        print("No hotel found")
        hotel_df.loc[index, "Hotel Name"] = "No hotel in range"

Fetching São Filipe
Fetching Kindia
Fetching Jojoima
No hotel found
Fetching Dingle
Fetching Boueni
Fetching Nong Kung Si
Fetching Bouloupari
Fetching Oussouye
Fetching Kampot


In [9]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
12,São Filipe,14.8961,-24.4956,72.91,69,0,7.43,CV,1611518357,Tortuga B&B
66,Kindia,10.0569,-12.8658,74.73,21,0,3.38,GN,1611518425,LES EAUX DE KILISSI
138,Jojoima,7.8792,-10.7878,75.61,34,0,2.89,SL,1611518559,No hotel in range
144,Dingle,10.9995,122.6711,74.91,92,0,6.78,PH,1611518560,SEGAYA PROPERTY
201,Boueni,-12.9025,45.0761,77.00,88,0,2.30,YT,1611518634,The feet in water
250,Nong Kung Si,16.6500,103.3000,73.00,99,0,3.85,TH,1611518703,บ้าน พี่บิ๊ก
343,Bouloupari,-21.8667,166.0500,78.80,73,0,6.91,NC,1611518838,Oua-Ya Lodge
520,Oussouye,12.4850,-16.5469,78.80,65,0,7.85,SN,1611519522,Auberge du Routard
550,Kampot,10.6167,104.1833,75.20,88,0,2.30,KH,1611519527,Two Moons Hotel


In [10]:
# Dropping out city with no hotel in range
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel in range"]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
12,São Filipe,14.8961,-24.4956,72.91,69,0,7.43,CV,1611518357,Tortuga B&B
66,Kindia,10.0569,-12.8658,74.73,21,0,3.38,GN,1611518425,LES EAUX DE KILISSI
144,Dingle,10.9995,122.6711,74.91,92,0,6.78,PH,1611518560,SEGAYA PROPERTY
201,Boueni,-12.9025,45.0761,77.00,88,0,2.30,YT,1611518634,The feet in water
250,Nong Kung Si,16.6500,103.3000,73.00,99,0,3.85,TH,1611518703,บ้าน พี่บิ๊ก
343,Bouloupari,-21.8667,166.0500,78.80,73,0,6.91,NC,1611518838,Oua-Ya Lodge
520,Oussouye,12.4850,-16.5469,78.80,65,0,7.85,SN,1611519522,Auberge du Routard
550,Kampot,10.6167,104.1833,75.20,88,0,2.30,KH,1611519527,Two Moons Hotel


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Adding pins to heatmap 
new_marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(new_marker)
fig

Figure(layout=FigureLayout(height='420px'))